In [3]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import shutil
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize

notebook_name = "gridrad_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T18:00:00"
end = "2010-01-20T19:00:00"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options
)

lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, longitude_range=lon_range, latitude_range=lat_range
)
args_dict = {"start": start, "end": end, "data_format": "single-levels"}
args_dict.update({"longitude_range": lon_range, "latitude_range": lat_range})
era5_sl_options = data.era5.data_options(**args_dict)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="gridrad_era5")

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="gridrad_geographic")

# Create the track_options dictionary
track_options = option.mcs(
    dataset="gridrad",
    global_flow_margin=70,
    unique_global_flow=False,
)
# Disable tracking/matching for cell and anvil objects
track_options[0]["cell"]["tracking"] = {"method": None}
track_options[0]["anvil"]["tracking"] = {"method": None}
option.check_options(track_options)
option.save_track_options(track_options, filename="gridrad_mcs")

# Create the display_options dictionary
# cell_vis_options = visualize.option.runtime_options(
#     "cell", save=True, style="presentation", figure_types=["mask"]
# )
# anvil_vis_options = visualize.option.runtime_options(
#     "anvil", save=True, style="presentation", figure_types=["mask"]
# )
# mcs_vis_options = visualize.option.runtime_options(
#     "mcs", save=True, style="presentation", figure_types=["mask", "match"]
# )
# visualize_options = {
#     "cell": cell_vis_options,
#     "anvil": anvil_vis_options,
#     "mcs": mcs_vis_options,
# }
visualize_options = None
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")

2024-10-07 18:25:19,290 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/data/gridrad_era5.yml
2024-10-07 18:25:19,302 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-07 18:25:19,304 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid/gridrad_geographic.yml
2024-10-07 18:25:19,308 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/track/gridrad_mcs.yml
2024-10-07 18:25:19,318 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/visualize/runtime_mcs.yml


In [12]:
output_directory = base_local / "runs/gridrad_demo"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["gridrad"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=base_local / "runs/gridrad_demo",
)


2024-10-07 18:14:09,045 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo.
2024-10-07 18:14:09,047 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-07 18:14:09,096 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-10-07 18:14:09,098 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-10-07 18:14:09,099 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-10-07 18:14:09,180 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-07 18:14:09,729 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-07 18:14:09,788 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-07 18:14:10,018 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-10-07 18:14:10,253 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-07 18:14:10,480 - thor.data.g

KeyboardInterrupt: 